In [1]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests

In [2]:
page_link ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page_response = requests.get(page_link, timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")

In [3]:
rows = page_content.find(class_='wikitable').tbody.find_all('tr')
body = rows[1:]

In [4]:
def extract_data(tr):
    return [col.get_text().strip() for col in tr.find_all('td')]
data = pd.DataFrame([extract_data(row) for row in body])

In [5]:
data.columns = ['PostalCode','Borough','Neighbourhood']
data=data[data['Borough']!='Not assigned']
Neighbourhood_Not_assigned = data['Neighbourhood']=='Not assigned'
data.loc[Neighbourhood_Not_assigned,'Neighbourhood'] = data[Neighbourhood_Not_assigned]['Borough']
data=data.groupby(['PostalCode','Borough']).aggregate({'Neighbourhood':[', '.join,'count']}).reset_index()
data.columns = ['PostalCode','Borough','NeighbourhoodNames','NeighbourhoodCount']

had done my work and no more assumption

In [6]:
data.shape

(103, 4)

In [7]:
geospatial = pd.read_csv('http://cocl.us/Geospatial_data')
geospatial.columns = ['PostalCode','Latitude','Longitude']

In [8]:
data = pd.merge(data,geospatial,how='left',on='PostalCode')

In [9]:
data.head()
data.describe()

,NeighbourhoodCount,Latitude,Longitude
count,103.000000,103.000000,103.000000
mean,2.058252,43.704608,-79.397153
std,1.406046,0.052463,0.097146
min,1.000000,43.602414,-79.615819
25%,1.000000,43.660567,-79.464763
50%,2.000000,43.696948,-79.388790
75%,3.000000,43.745320,-79.340923
max,8.000000,43.836125,-79.160497


In [10]:
#!pip install folium
import folium

# create map of Toronto using latitude and longitude values
latitude, longitude = data['Latitude'].mean(), data['Longitude'].mean()
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

I suspect that second digit of postal code might have some meaning that it will be located nearby, so I plotted 1 to 9 as different colors, and they appear as clusters.

In [11]:
colors = ['#ff0000','#ff8000','#ffff00','#00ff00','#00ffff','#0080ff','#0000ff','#8000ff','#ff00ff']

# add markers to map
for lat, lng, borough, neighborhood, count, code2, code3 in zip(data['Latitude'], data['Longitude'], data['Borough'], data['NeighbourhoodNames'], data['NeighbourhoodCount'],data['PostalCode'].str.slice(1,2).astype(np.int),(data['PostalCode'].str.slice(2,3).map(ord)-65)/26):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=code3*10,
        popup=label,
        color=colors[code2-1],
        fill=True,
        fill_color=colors[code2-1],
        fill_opacity=0.7,
    ).add_to(map_toronto)  
    
map_toronto

Plotted The third digit of postal code A-Z as different pin sizes, they somehow imply direction on the map.